In [ ]:
# !pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.2/434.2 MB 5.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 12.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-4.0.1-py2.py3-none-any.whl size=434813791 sha256=e353c55ca48bcf6ac172c5ded00de89d70b22c3ffa14d4de85020d17cc561d10
  Stored in directory: /Users/krishan/Library/Caches/pip/wheels/76/f8/dc/9195b82b8586561710077d42370ae400e8a023af43052d1fec
Successfully built pyspark

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [7]:
from pyspark.sql import SparkSession

# Stop any existing SparkSession to ensure a clean start.
# This helps prevent issues like "Another SparkContext is being constructed"
# and ensures getOrCreate() can either retrieve a clean one or create a new one successfully.
# It also helps mitigate Spark UI port binding issues if previous sessions
# weren't properly shut down.
existing_spark = SparkSession.getActiveSession()
if existing_spark:
    print("Stopping existing SparkSession to ensure a clean restart.")
    existing_spark.stop()

# Start spark session
# To address the 'java.net.BindException: Can't assign requested address' error,
# explicitly set spark.driver.bindAddress and spark.driver.host to '127.0.0.1' (localhost).
# This ensures Spark binds to the loopback interface, which is typically available
# and helps avoid network configuration issues in local environments.
spark = SparkSession.builder \
    .appName("PySpark") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .config("spark.driver.host", "127.0.0.1") \
    .getOrCreate()

# Check spark version
print(spark.version)


4.0.1


25/11/02 02:33:12 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:59)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:75)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:53)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:502)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:486)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.command

# DATA READING

### Data Reading JSON

In [8]:
df_json = spark.read.format('json').option('inferSchema',True)\
                    .option('header',True)\
                    .option('multiLine',False)\
                    .load('drivers.json')

In [9]:
df_json.show()

+----+----------+--------+----------+--------------------+-----------+------+--------------------+
|code|       dob|driverId| driverRef|                name|nationality|number|                 url|
+----+----------+--------+----------+--------------------+-----------+------+--------------------+
| HAM|1985-01-07|       1|  hamilton|   {Lewis, Hamilton}|    British|    44|http://en.wikiped...|
| HEI|1977-05-10|       2|  heidfeld|    {Nick, Heidfeld}|     German|    \N|http://en.wikiped...|
| ROS|1985-06-27|       3|   rosberg|     {Nico, Rosberg}|     German|     6|http://en.wikiped...|
| ALO|1981-07-29|       4|    alonso|  {Fernando, Alonso}|    Spanish|    14|http://en.wikiped...|
| KOV|1981-10-19|       5|kovalainen|{Heikki, Kovalainen}|    Finnish|    \N|http://en.wikiped...|
| NAK|1985-01-11|       6|  nakajima|  {Kazuki, Nakajima}|   Japanese|    \N|http://en.wikiped...|
| BOU|1979-02-28|       7|  bourdais|{Sébastien, Bourd...|     French|    \N|http://en.wikiped...|
| RAI|1979

### Data Reading Utils

In [0]:
# dbutils.fs.ls('/FileStore/tables/')

In [64]:
df = spark.read.format('csv').option('inferSchema',True).option('header',True)\
    .load('BigMart Sales.csv')

In [15]:
df.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Superma

### Schema Definition

In [16]:
df.printSchema()

root
 |-- Item_Identifier: string (nullable = true)
 |-- Item_Weight: double (nullable = true)
 |-- Item_Fat_Content: string (nullable = true)
 |-- Item_Visibility: double (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Item_MRP: double (nullable = true)
 |-- Outlet_Identifier: string (nullable = true)
 |-- Outlet_Establishment_Year: integer (nullable = true)
 |-- Outlet_Size: string (nullable = true)
 |-- Outlet_Location_Type: string (nullable = true)
 |-- Outlet_Type: string (nullable = true)
 |-- Item_Outlet_Sales: double (nullable = true)



### DDL SCHEMA

In [60]:
my_ddl_schema = '''
                    Item_Identifier STRING,
                    Item_Weight STRING,
                    Item_Fat_Content STRING, 
                    Item_Visibility DOUBLE,
                    Item_Type STRING,
                    Item_MRP DOUBLE,
                    Outlet_Identifier STRING,
                    Outlet_Establishment_Year INT,
                    Outlet_Size STRING,
                    Outlet_Location_Type STRING, 
                    Outlet_Type STRING,
                    Item_Outlet_Sales DOUBLE
                
                ''' 

In [61]:
df = spark.read.format('csv')\
            .schema(my_ddl_schema)\
            .option('header',True)\
            .load('BigMart Sales.csv') 

In [62]:
df.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Superma

In [38]:
df.printSchema()

root
 |-- Item_Identifier: string (nullable = true)
 |-- Item_Weight: string (nullable = true)
 |-- Item_Fat_Content: string (nullable = true)
 |-- Item_Visibility: double (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Item_MRP: double (nullable = true)
 |-- Outlet_Identifier: string (nullable = true)
 |-- Outlet_Establishment_Year: integer (nullable = true)
 |-- Outlet_Size: string (nullable = true)
 |-- Outlet_Location_Type: string (nullable = true)
 |-- Outlet_Type: string (nullable = true)
 |-- Item_Outlet_Sales: double (nullable = true)
 |-- random_col: string (nullable = true)



### StructType() Schema

In [39]:
from pyspark.sql.types import * 
from pyspark.sql.functions import *  

In [57]:
my_strct_schema = StructType([
                                StructField('Item_Identifier',StringType(),True),
                                StructField('Item_Weight',DoubleType(),True),
                                StructField('Item_Fat_Content',StringType(),True),
                                StructField('Item_Visibility',StringType(),True),
                                StructField('Item_MRP',StringType(),True),
                                StructField('Outlet_Identifier',StringType(),True),
                                StructField('Outlet_Establishment_Year',StringType(),True),
                                StructField('Outlet_Size',StringType(),True),
                                StructField('Outlet_Location_Type',StringType(),True),
                                StructField('Outlet_Type',StringType(),True),
                                StructField('Item_Outlet_Sales',StringType(),True)

])

In [42]:
df = spark.read.format('csv')\
            .schema(my_strct_schema)\
            .option('header',True)\
            .load('BigMart Sales.csv')

In [43]:
df.printSchema()

root
 |-- Item_Identifier: string (nullable = true)
 |-- Item_Weight: string (nullable = true)
 |-- Item_Fat_Content: string (nullable = true)
 |-- Item_Visibility: string (nullable = true)
 |-- Item_MRP: string (nullable = true)
 |-- Outlet_Identifier: string (nullable = true)
 |-- Outlet_Establishment_Year: string (nullable = true)
 |-- Outlet_Size: string (nullable = true)
 |-- Outlet_Location_Type: string (nullable = true)
 |-- Outlet_Type: string (nullable = true)
 |-- Item_Outlet_Sales: string (nullable = true)



# TRANSFORMATIONS

### SELECT 

In [44]:
df.show()

+---------------+-----------+----------------+---------------+--------------------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|            Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|         249.8092|                   OUT049|       1999|              Medium|     Tier 1|Supermarket Type1|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks|          48.2692|                   OUT018|       2009|              Medium|     Tier 3|Supermarket Type2|
|          FDN15|       17.5|         Low Fat

25/11/02 02:40:37 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 12, schema size: 11
CSV file: file:///Users/krishan/Documents/GitHub/PySpark-APIs/BigMart%20Sales.csv


In [46]:
df.select(col('Item_Identifier'),col('Item_Weight'),col('Item_Fat_Content')).show()

+---------------+-----------+----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|
+---------------+-----------+----------------+
|          FDA15|        9.3|         Low Fat|
|          DRC01|       5.92|         Regular|
|          FDN15|       17.5|         Low Fat|
|          FDX07|       19.2|         Regular|
|          NCD19|       8.93|         Low Fat|
|          FDP36|     10.395|         Regular|
|          FDO10|      13.65|         Regular|
|          FDP10|       NULL|         Low Fat|
|          FDH17|       16.2|         Regular|
|          FDU28|       19.2|         Regular|
|          FDY07|       11.8|         Low Fat|
|          FDA03|       18.5|         Regular|
|          FDX32|       15.1|         Regular|
|          FDS46|       17.6|         Regular|
|          FDF32|      16.35|         Low Fat|
|          FDP49|          9|         Regular|
|          NCB42|       11.8|         Low Fat|
|          FDP49|          9|         Regular|
|          DR

### ALIAS

In [47]:
df.select(col('Item_Identifier').alias('Item_ID')).show()

+-------+
|Item_ID|
+-------+
|  FDA15|
|  DRC01|
|  FDN15|
|  FDX07|
|  NCD19|
|  FDP36|
|  FDO10|
|  FDP10|
|  FDH17|
|  FDU28|
|  FDY07|
|  FDA03|
|  FDX32|
|  FDS46|
|  FDF32|
|  FDP49|
|  NCB42|
|  FDP49|
|  DRI11|
|  FDU02|
+-------+
only showing top 20 rows


In [51]:
df.show()

+---------------+-----------+----------------+---------------+--------------------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|            Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|         249.8092|                   OUT049|       1999|              Medium|     Tier 1|Supermarket Type1|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks|          48.2692|                   OUT018|       2009|              Medium|     Tier 3|Supermarket Type2|
|          FDN15|       17.5|         Low Fat

25/11/02 02:44:46 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 12, schema size: 11
CSV file: file:///Users/krishan/Documents/GitHub/PySpark-APIs/BigMart%20Sales.csv


### FILTER

#### Scenario - 1 

In [49]:
df.filter(col('Item_Fat_Content')=='Regular').show()

+---------------+-----------+----------------+---------------+--------------------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|            Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks|          48.2692|                   OUT018|       2009|              Medium|     Tier 3|Supermarket Type2|
|          FDX07|       19.2|         Regular|              0|Fruits and Vegeta...|          182.095|                   OUT010|       1998|                NULL|     Tier 3|    Grocery Store|
|          FDP36|     10.395|         Regular

25/11/02 02:44:15 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 12, schema size: 11
CSV file: file:///Users/krishan/Documents/GitHub/PySpark-APIs/BigMart%20Sales.csv


#### Scenario - 2

In [65]:
df.filter((col('Item_Fat_Content') == 'Regular') & (col('Item_Weight') < 10)).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Supermarket Type2|         443.4228|
|          FDP49|        9.0|         Regular|    0.069088961|           Breakfast| 56.3614|           OUT046|                     1997|      Small|              Tier 1|Superma

#### Scenario - 3

In [66]:
df.filter((col('Outlet_Size').isNull()) & (col('Outlet_Location_Type').isin('Tier 1','Tier 2'))).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          FDH17|       16.2|         Regular|    0.016687114|        Frozen Foods| 96.9726|           OUT045|                     2002|       NULL|              Tier 2|Supermarket Type1|        1076.5986|
|          FDU28|       19.2|         Regular|     0.09444959|        Frozen Foods|187.8214|           OUT017|                     2007|       NULL|              Tier 2|Superma

### withColumnRenamed

In [67]:
df.withColumnRenamed('Item_Weight','Item_Wt').show()

+---------------+-------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Wt|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          FDA15|    9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|
|          DRC01|   5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Supermarket Type2|         

### withColumn

#### Scenario - 1

In [68]:
df = df.withColumn('flag',lit("new")) 

In [69]:
df.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138| new|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|        

In [70]:
df.withColumn('multiply',col('Item_Weight')*col('Item_MRP')).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+------------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|          multiply|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+------------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138| new|2323.2255600000003|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drin

#### Scenario - 2

In [71]:
df = df.withColumn('Item_Fat_Content',regexp_replace(col('Item_Fat_Content'),"Regular","Reg"))\
    .withColumn('Item_Fat_Content',regexp_replace(col('Item_Fat_Content'),"Low Fat","Lf"))

df.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          FDA15|        9.3|              Lf|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138| new|
|          DRC01|       5.92|             Reg|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|        

### Type Casting

In [72]:
df = df.withColumn('Item_Weight', col('Item_Weight').cast(StringType())) 

In [73]:
df.printSchema()

root
 |-- Item_Identifier: string (nullable = true)
 |-- Item_Weight: string (nullable = true)
 |-- Item_Fat_Content: string (nullable = true)
 |-- Item_Visibility: double (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Item_MRP: double (nullable = true)
 |-- Outlet_Identifier: string (nullable = true)
 |-- Outlet_Establishment_Year: integer (nullable = true)
 |-- Outlet_Size: string (nullable = true)
 |-- Outlet_Location_Type: string (nullable = true)
 |-- Outlet_Type: string (nullable = true)
 |-- Item_Outlet_Sales: double (nullable = true)
 |-- flag: string (nullable = false)



### sort

#### Scenario - 1

In [74]:
df.sort(col('Item_Weight').desc()).show()

+---------------+-----------+----------------+---------------+------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|   Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          FDR13|      9.895|             Reg|    0.028696932|      Canned|117.0492|           OUT013|                     1987|       High|              Tier 3|Supermarket Type1|         810.9444| new|
|          DRD49|      9.895|              Lf|    0.167799329| Soft Drinks|239.4564|           OUT035|                     2004|      Small|              Tier 2|Supermarket Type1|        5

#### Scenario - 2

In [75]:
df.sort(col('Item_Visibility').asc()).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          DRB48|      16.75|             Reg|            0.0|         Soft Drinks| 39.3822|           OUT046|                     1997|      Small|              Tier 1|Supermarket Type1|         353.5398| new|
|          FDS32|      17.75|             Reg|            0.0|Fruits and Vegeta...|139.9838|           OUT045|                     2002|       NULL|        

#### Scenario - 3

In [76]:
df.sort(['Item_Weight','Item_Visibility'],ascending = [0,0]).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          DRD49|      9.895|              Lf|    0.168780385|         Soft Drinks|236.8564|           OUT017|                     2007|       NULL|              Tier 2|Supermarket Type1|         4767.128| new|
|          DRD49|      9.895|              Lf|     0.16817143|         Soft Drinks|237.7564|           OUT045|                     2002|       NULL|        

#### Scenario - 4

In [77]:
df.sort(['Item_weight','Item_Visibility'], ascending = [0,1]).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          FDR13|      9.895|             Reg|    0.028696932|              Canned|117.0492|           OUT013|                     1987|       High|              Tier 3|Supermarket Type1|         810.9444| new|
|          FDR13|      9.895|             Reg|    0.028765486|              Canned|115.3492|           OUT049|                     1999|     Medium|        

### Limit

In [78]:
df.limit(10).show() 

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          FDA15|        9.3|              Lf|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138| new|
|          DRC01|       5.92|             Reg|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|        

### DROP

#### Scenario-1

In [79]:
df.drop('Item_Visibility').show()

+---------------+-----------+----------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          FDA15|        9.3|              Lf|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138| new|
|          DRC01|       5.92|             Reg|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Supermarket Type2|         443.4228| new|
|          FDN15|       1

#### Scenario-2

In [80]:
df.drop('Item_Visibility','Item_Type').show()

+---------------+-----------+----------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          FDA15|        9.3|              Lf|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138| new|
|          DRC01|       5.92|             Reg| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Supermarket Type2|         443.4228| new|
|          FDN15|       17.5|              Lf| 141.618|           OUT049|                     1999|     Medium|              Tier 

### DRop_Duplicates

In [81]:
df.dropDuplicates().show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----+
|          FDF08|       14.3|             Reg|    0.065207559|Fruits and Vegeta...| 89.8856|           OUT046|                     1997|      Small|              Tier 1|Supermarket Type1|         966.7416| new|
|          FDR46|      16.85|              Lf|    0.139417654|         Snack Foods| 147.476|           OUT046|                     1997|      Small|        

#### Scenario - 2

In [0]:
df.drop_duplicates(subset=['Item_Type']).show()

In [0]:
df.distinct().show()

### UNION and UNION BY NAME

####Preaparing Dataframes

In [0]:
data1 = [('1','kad'),
        ('2','sid')]
schema1 = 'id STRING, name STRING' 

df1 = spark.createDataFrame(data1,schema1)

data2 = [('3','rahul'),
        ('4','jas')]
schema2 = 'id STRING, name STRING' 

df2 = spark.createDataFrame(data2,schema2)



In [0]:
df1.show()

In [0]:
df2.show()

### Union

In [0]:
df1.union(df2).show()

In [0]:
data1 = [('kad','1',),
        ('sid','2',)]
schema1 = 'name STRING, id STRING' 

df1 = spark.createDataFrame(data1,schema1)

df1.show()

In [0]:
df1.union(df2).show()

### Union by Name

In [0]:
df1.unionByName(df2).show()

### String Functions

#### Initcap()

In [0]:
df.select(upper('Item_Type').alias('upper_Item_Type')).show()

### Date Functions

#### Current_Date

In [0]:
df = df.withColumn('curr_date',current_date())

df.show()

#### Date_Add()

In [0]:
df = df.withColumn('week_after',date_add('curr_date',7))

df.show()

#### Date_Sub()

In [0]:
df.withColumn('week_before',date_sub('curr_date',7)).show()

In [0]:
df = df.withColumn('week_before',date_add('curr_date',-7)) 

df.show()

### DateDIFF

In [0]:
df = df.withColumn('datediff',datediff('week_after','curr_date'))

df.show()

### Date_Format()

In [0]:
df = df.withColumn('week_before',date_format('week_before','dd-MM-yyyy'))

df.show()

### Handling Nulls

#### Dropping NUlls

In [0]:
df.dropna('all').show()

In [0]:
df.dropna('any').show()

In [0]:
df.dropna(subset=['Outlet_Size']).show()

In [0]:
df.show()

#### Filling Nulls

In [0]:
df.fillna('NotAvailable').show()

In [0]:
df.fillna('NotAvailable',subset=['Outlet_Size']).show()

### SPLIT and Indexing

#### SPLIT

In [0]:
df.withColumn('Outlet_Type',split('Outlet_Type',' ')).show()

#### Indexing

In [0]:
df.withColumn('Outlet_Type',split('Outlet_Type',' ')[1]).show()

### Explode

In [0]:
df_exp = df.withColumn('Outlet_Type',split('Outlet_Type',' '))

df_exp.show()

In [0]:
df_exp.withColumn('Outlet_Type',explode('Outlet_Type')).show()

In [0]:
df_exp.show()

In [0]:
df_exp.withColumn('Type1_flag',array_contains('Outlet_Type','Type1')).show()

### GroupBY

#### Scenario - 1

In [0]:
df.show()

In [0]:
df.groupBy('Item_Type').agg(sum('Item_MRP')).show()

#### Scenario - 2

In [0]:
df.groupBy('Item_Type').agg(avg('Item_MRP')).show()

#### SCenario - 3

In [0]:
df.groupBy('Item_Type','Outlet_Size').agg(sum('Item_MRP').alias('Total_MRP')).show()

#### Scenario - 4

In [0]:
df.groupBy('Item_Type','Outlet_Size').agg(sum('Item_MRP'),avg('Item_MRP')).show()

### Collect_List

In [0]:
data = [('user1','book1'),
        ('user1','book2'),
        ('user2','book2'),
        ('user2','book4'),
        ('user3','book1')]

schema = 'user string, book string'

df_book = spark.createDataFrame(data,schema)

df_book.show()

In [0]:
df_book.groupBy('user').agg(collect_list('book')).show()

In [0]:
df.select('Item_Type','Outlet_Size','Item_MRP').show()

### PIVOT

In [0]:
df.groupBy('Item_Type').pivot('Outlet_Size').agg(avg('Item_MRP')).show()

### When-Otherwise

#### Scenario - 1

In [0]:
df = df.withColumn('veg_flag',when(col('Item_Type')=='Meat','Non-Veg').otherwise('Veg'))

In [0]:
df.show()

In [0]:
df.withColumn('veg_exp_flag',when(((col('veg_flag')=='Veg') & (col('Item_MRP')<100)),'Veg_Inexpensive')\
                            .when((col('veg_flag')=='Veg') & (col('Item_MRP')>100),'Veg_Expensive')\
                            .otherwise('Non_Veg')).show() 

### JOINS

In [0]:
dataj1 = [('1','gaur','d01'),
          ('2','kit','d02'),
          ('3','sam','d03'),
          ('4','tim','d03'),
          ('5','aman','d05'),
          ('6','nad','d06')] 

schemaj1 = 'emp_id STRING, emp_name STRING, dept_id STRING' 

df1 = spark.createDataFrame(dataj1,schemaj1)

dataj2 = [('d01','HR'),
          ('d02','Marketing'),
          ('d03','Accounts'),
          ('d04','IT'),
          ('d05','Finance')]

schemaj2 = 'dept_id STRING, department STRING'

df2 = spark.createDataFrame(dataj2,schemaj2)

In [0]:
df1.show()

In [0]:
df2.show()

#### Inner Join

In [0]:
df1.join(df2, df1['dept_id']==df2['dept_id'],'inner').show()

#### Left Join

In [0]:
df1.join(df2,df1['dept_id']==df2['dept_id'],'left').show()

#### LEFT JOIN

In [0]:
df1.join(df2,df1['dept_id']==df2['dept_id'],'right').show()

#### ANTI JOIN 

In [0]:
df1.join(df2,df1['dept_id']==df2['dept_id'],'anti').show()

### WINDOW FUNCTIONS

#### ROW_NUMBER()

In [0]:
df.show()

In [0]:
from pyspark.sql.window import Window

In [0]:
df.withColumn('rowCol',row_number().over(Window.orderBy('Item_Identifier'))).show()

#### RANK VS DENSE RANK 

In [0]:
df.withColumn('rank',rank().over(Window.orderBy(col('Item_Identifier').desc())))\
        .withColumn('denseRank',dense_rank().over(Window.orderBy(col('Item_Identifier').desc()))).show()

In [0]:
df.withColumn('dum',sum('Item_MRP').over(Window.orderBy('Item_Identifier').rowsBetween(Window.unboundedPreceding,Window.currentRow))).show()

#### Cumulative Sum

In [0]:
df.withColumn('cumsum',sum('Item_MRP').over(Window.orderBy('Item_Type'))).show()

In [0]:
df.withColumn('cumsum',sum('Item_MRP').over(Window.orderBy('Item_Type').rowsBetween(Window.unboundedPreceding,Window.currentRow))).show()

In [0]:
df.withColumn('totalsum',sum('Item_MRP').over(Window.orderBy('Item_Type').rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing))).show()

### USER DEFINED FUNCTIONS (UDF)

#### STEP - 1

In [0]:
def my_func(x):
    return x*x 

#### STEP - 2

In [0]:
my_udf = udf(my_func)

In [0]:
df.withColumn('mynewcol',my_udf('Item_MRP')).show()

### DATA WRITING

#### CSV

In [0]:
df.write.format('csv')\
        .save('/FileStore/tables/CSV/data.csv')

#### APPEND

In [0]:
df.write.format('csv')\
        .mode('append')\
        .save('/FileStore/tables/CSV/data.csv')

In [0]:
df.write.format('csv')\
        .mode('append')\
        .option('path','/FileStore/tables/CSV/data.csv')\
        .save()

#### Overwrite

In [0]:
df.write.format('csv')\
.mode('overwrite')\
.option('path','/FileStore/tables/CSV/data.csv')\
.save()

#### Error

In [0]:
df.write.format('csv')\
.mode('error')\
.option('path','/FileStore/tables/CSV/data.csv')\
.save()

#### Ignore

In [0]:
df.write.format('csv')\
.mode('ignore')\
.option('path','/FileStore/tables/CSV/data.csv')\
.save()

#### PARQUET

In [0]:
df.write.format('parquet')\
.mode('overwrite')\
.option('path','/FileStore/tables/CSV/data.csv')\
.save()

#### TABLE

In [0]:
df.write.format('parquet')\
.mode('overwrite')\
.saveAsTable('my_table')

In [0]:
df.show()

### SPARK SQL

#### createTempView 

In [0]:
df.createTempView('my_view')

In [0]:
%sql

select * from my_view where Item_Fat_Content = 'Lf'

In [0]:
df_sql = spark.sql("select * from my_view where Item_Fat_Content = 'Lf'")

In [0]:
df_sql.show()